In [11]:
import pandas as pd
# Retrieve all documents from the collection and convert them to a Pandas DataFrame
cursor = collection.find({})
df = pd.DataFrame(list(cursor))

# Display the DataFrame
display(df.head(2))

,_id,arrest_date,arrest_time,sex,race,searched,reason_stopped,search_based_on,search_found,race_known,lng,lat
0,655a73c0e3a3e10a808d9f8b,1.420070e+12,0,M,HISPANIC OR LATINO,YES = 1,CALL FOR SERVICE,INCIDENTAL TO ARREST,NOTHING,NO - RACE OR ETHNICITY WAS NOT KNOWN BEFORE STOP,7.181465,85.773044
1,655a73c0e3a3e10a808d9f8c,1.420070e+12,0,M,WHITE,YES = 1,VIOLATION OF TRANSPORTATION CODE/VEHICLE LAWS,INCIDENTAL TO ARREST,DRUGS,NO - RACE OR ETHNICITY WAS NOT KNOWN BEFORE STOP,6.341423,85.781942


In [1]:
 # Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import pandas as pd
import tensorflow as tf



from sklearn.preprocessing import LabelEncoder
import numpy as np


In [10]:
# Import our input dataset
df = pd.read_csv('https://profilingbucket.s3.us-east-2.amazonaws.com/combined_cleaned_data.csv')
df.head()

,arrest_date,arrest_time,sex,race,searched,reason_stopped,search_based_on,search_found,race_known,lng,lat
0,2015-01-01,1970-01-01 00:00:00.000000002,M,HISPANIC OR LATINO,YES = 1,CALL FOR SERVICE,INCIDENTAL TO ARREST,NOTHING,NO - RACE OR ETHNICITY WAS NOT KNOWN BEFORE STOP,7.181465,85.773044
1,2015-01-01,1970-01-01 00:00:00.000000317,M,WHITE,YES = 1,VIOLATION OF TRANSPORTATION CODE/VEHICLE LAWS,INCIDENTAL TO ARREST,DRUGS,NO - RACE OR ETHNICITY WAS NOT KNOWN BEFORE STOP,6.341423,85.781942
2,2015-01-01,1970-01-01 00:00:00.000000317,F,WHITE,YES = 1,VIOLATION OF TRANSPORTATION CODE/VEHICLE LAWS,INCIDENTAL TO ARREST,DRUGS,NO - RACE OR ETHNICITY WAS NOT KNOWN BEFORE STOP,6.341423,85.781942
3,2015-01-01,1970-01-01 00:00:00.000000426,M,WHITE,YES = 1,VIOLATION OF TRANSPORTATION CODE/VEHICLE LAWS,INCIDENTAL TO ARREST,NOTHING,NO - RACE OR ETHNICITY WAS NOT KNOWN BEFORE STOP,7.072722,85.799348
4,2015-01-01,1970-01-01 00:00:00.000000106,F,WHITE,YES = 1,CONSENSUAL CONTACT,INCIDENTAL TO ARREST,OTHER,NO - RACE OR ETHNICITY WAS NOT KNOWN BEFORE STOP,7.134598,85.777384


In [3]:
# columns_to_drop = ['arrest_date', 'arrest_time', 'lng', 'lat']
# df = df.drop(columns=columns_to_drop)

In [4]:
# Preprocess categorical variables
df_processed = pd.get_dummies(df, columns=['sex', 'race',])

# Extract features from date and time
df_processed['arrest_date'] = pd.to_datetime(df_processed['arrest_date'])
df_processed['day_of_week'] = df_processed['arrest_date'].dt.dayofweek
df_processed['hour'] = pd.to_datetime(df_processed['arrest_time']).dt.hour

booleans = ['sex_F','sex_M', 'sex_U', 'sex_not_listed', 'race_AMERICAN INDIAN/ALASKAN NATIVE', 'race_ASIAN', 'race_BLACK',
            'race_HAWAIIAN/PACIFIC ISLANDER', 'race_HISPANIC OR LATINO', 'race_MIDDLE EASTERN', 'race_UNKNOWN', 'race_WHITE',
            'race_not_listed','day_of_week', 'hour']

# Convert 'YES = 1' to 1 and other values to 0 in the 'searched' column
df_processed['searched'] = df_processed['searched'].map(lambda x: 1 if 'YES' in str(x).upper() else 0)

# Convert 'searched' column to integer
df_processed['searched'] = df_processed['searched'].astype(int)

# Convert other boolean columns to integers if applicable
df_processed[booleans] = df_processed[booleans].astype(int)

# Drop original datetime columns
df_processed = df_processed.drop(['arrest_date', 'arrest_time'], axis=1)

# Define X and y
y = df_processed['searched']
X = df_processed.drop(columns=['searched'])


In [5]:
X

,reason_stopped,search_based_on,search_found,race_known,lng,lat,sex_F,sex_M,sex_U,sex_not_listed,...,race_ASIAN,race_BLACK,race_HAWAIIAN/PACIFIC ISLANDER,race_HISPANIC OR LATINO,race_MIDDLE EASTERN,race_UNKNOWN,race_WHITE,race_not_listed,day_of_week,hour
0,CALL FOR SERVICE,INCIDENTAL TO ARREST,NOTHING,NO - RACE OR ETHNICITY WAS NOT KNOWN BEFORE STOP,7.181465,85.773044,0,1,0,0,...,0,0,0,1,0,0,0,0,3,0
1,VIOLATION OF TRANSPORTATION CODE/VEHICLE LAWS,INCIDENTAL TO ARREST,DRUGS,NO - RACE OR ETHNICITY WAS NOT KNOWN BEFORE STOP,6.341423,85.781942,0,1,0,0,...,0,0,0,0,0,0,1,0,3,0
2,VIOLATION OF TRANSPORTATION CODE/VEHICLE LAWS,INCIDENTAL TO ARREST,DRUGS,NO - RACE OR ETHNICITY WAS NOT KNOWN BEFORE STOP,6.341423,85.781942,1,0,0,0,...,0,0,0,0,0,0,1,0,3,0
3,VIOLATION OF TRANSPORTATION CODE/VEHICLE LAWS,INCIDENTAL TO ARREST,NOTHING,NO - RACE OR ETHNICITY WAS NOT KNOWN BEFORE STOP,7.072722,85.799348,0,1,0,0,...,0,0,0,0,0,0,1,0,3,0
4,CONSENSUAL CONTACT,INCIDENTAL TO ARREST,OTHER,NO - RACE OR ETHNICITY WAS NOT KNOWN BEFORE STOP,7.134598,85.777384,1,0,0,0,...,0,0,0,0,0,0,1,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41418,Violation of law other than traffic,PROBABLE CAUSE,NOTHING,YES - RACE OR ETHNICITY WAS KNOWN BEFORE STOP,7.335770,85.798170,0,1,0,0,...,0,0,0,0,0,0,1,0,5,0
41419,Violation of law other than traffic,INCIDENTAL TO ARREST,NOTHING,YES - RACE OR ETHNICITY WAS KNOWN BEFORE STOP,7.071786,85.880118,0,1,0,0,...,0,0,0,0,0,0,1,0,4,0
41420,Violation of law other than traffic,INCIDENTAL TO ARREST,NOTHING,YES - RACE OR ETHNICITY WAS KNOWN BEFORE STOP,8.688551,85.863456,0,1,0,0,...,0,0,0,0,0,0,1,0,3,0
41421,Violation of law other than traffic,INCIDENTAL TO ARREST,NOTHING,YES - RACE OR ETHNICITY WAS KNOWN BEFORE STOP,8.613016,85.849483,0,1,0,0,...,0,0,0,0,0,0,1,0,1,0


In [6]:
# booleans = ['sex_F','sex_M', 'sex_U', 'sex_not_listed', 'race_AMERICAN INDIAN/ALASKAN NATIVE', 'race_ASIAN', 'race_BLACK',
#             'race_HAWAIIAN/PACIFIC ISLANDER', 'race_HISPANIC OR LATINO', 'race_MIDDLE EASTERN', 'race_UNKNOWN', 'race_WHITE',
#             'race_not_listed','day_of_week', 'hour']

# # Convert 'YES = 1' to 1 and other values to 0 in the 'searched' column
# df_processed['searched'] = df_processed['searched'].map(lambda x: 1 if 'YES' in str(x).upper() else 0)

# # Convert 'searched' column to integer
# df_processed['searched'] = df_processed['searched'].astype(int)

# # Convert other boolean columns to integers if applicable
# df_processed[booleans] = df_processed[booleans].astype(int)

# # Define X and y
# y = df_processed['searched']
# X = df_processed.drop(columns=['searched'])

In [7]:
#  # Review the features data
# X.head()

In [8]:
# Now you can use X and y in a machine learning model
# For example, you can use scikit-learn to split the data into training and testing sets and train a model
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a RandomForestClassifier (you can choose a different model based on your needs)
model = RandomForestClassifier(random_state=42)

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

ValueError: could not convert string to float: 'Violation of law other than traffic'

In [9]:
spark.close()

NameError: name 'spark' is not defined